### 자 딥러닝 시작해 보자고~

- 일단 가상환경부터 만들어줌! 

환경쒝!

In [1]:
import tensorflow as tf, platform, pathlib
print("Python:", platform.python_version())
print("TF:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))
print("CWD:", pathlib.Path().resolve())

/Users/suakim/Documents/프로젝트/flutter 딥러닝 플젝/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Python: 3.9.6
TF: 2.16.1
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
CWD: /Users/suakim/Documents/프로젝트/flutter 딥러닝 플젝/Model


In [5]:
import pathlib, re, os, shutil

# 1) Oxford 루트 자동 탐색
def find_oxford_root():
    here = pathlib.Path().resolve()
    for p in [
        here / "data_sources" / "oxford",
        here.parent / "data_sources" / "oxford",
        here.parent.parent / "data_sources" / "oxford",
        here.parent.parent.parent / "data_sources" / "oxford",
    ]:
        if (p / "annotations" / "list.txt").exists() and (p / "images").exists():
            return p
    return None

root = find_oxford_root()
if root is None:
    raise FileNotFoundError("Oxford-IIIT Pet 경로를 못 찾았어요. data_sources/oxford 아래에 images/와 annotations/list.txt가 있어야 합니다.")

img_dir, ann_dir = root/"images", root/"annotations"
dest = pathlib.Path("data"); dest.mkdir(parents=True, exist_ok=True)

USE_HARDLINKS = True   # 같은 디스크면 용량 절약
count = 0; breeds = set()

with open(ann_dir/"list.txt") as f:
    for raw in f:
        # 2) 주석/여분 토큰 방어 코드
        line = raw.split('#', 1)[0].strip()   # # 뒤 주석 제거
        if not line:
            continue
        parts = line.split()
        if len(parts) < 4:
            continue
        name, cls_id, species, breed_id = parts[:4]  # 앞 4개만 사용

        # 3) 강아지(species==2)만 선별
        try:
            if int(species) != 2:   # 1=cat, 2=dog
                continue
        except ValueError:
            continue  # 비정상 줄 방어

        # 4) 파일명 → 품종명 추출 (맨 끝 _숫자 제거)
        breed = re.sub(r"_(\d+)$", "", name)
        src = img_dir / (name + ".jpg")
        if not src.exists():
            continue

        (dest / breed).mkdir(parents=True, exist_ok=True)
        dst = dest / breed / (name + ".jpg")
        try:
            if USE_HARDLINKS:
                if dst.exists(): dst.unlink()
                os.link(src, dst)
            else:
                shutil.copy2(src, dst)
        except OSError:
            shutil.copy2(src, dst)

        breeds.add(breed); count += 1

print(f"✅ dogs images: {count}, breeds: {len(breeds)}")
print("예시 품종 10개:", sorted(list(breeds))[:10])
print("Oxford root:", root)
print("Export dir:", dest.resolve())


✅ dogs images: 4978, breeds: 25
예시 품종 10개: ['american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'boxer', 'chihuahua', 'english_cocker_spaniel', 'english_setter', 'german_shorthaired', 'great_pyrenees']
Oxford root: /Users/suakim/Documents/프로젝트/flutter 딥러닝 플젝/data_sources/oxford
Export dir: /Users/suakim/Documents/프로젝트/flutter 딥러닝 플젝/Model/data


In [7]:
import pathlib
# Model 폴더에서 작업 중이면 "data", 루트면 "Model/data"
candidates = [pathlib.Path("data"), pathlib.Path("Model/data")]
for p in candidates:
    if p.exists() and any(p.iterdir()):
        DATA_DIR = p
        break
print("DATA_DIR:", DATA_DIR.resolve())


DATA_DIR: /Users/suakim/Documents/프로젝트/flutter 딥러닝 플젝/Model/data


## 품종마다 갯수 확인

In [8]:
import pathlib, collections
root = DATA_DIR
cnt = {d.name: len(list(d.glob("*.jpg"))) for d in root.iterdir() if d.is_dir()}
print("총 이미지:", sum(cnt.values()))
print("클래스 수:", len(cnt))
print("상위 10개:", sorted(cnt.items(), key=lambda x: -x[1])[:10])

총 이미지: 4978
클래스 수: 25
상위 10개: [('wheaten_terrier', 200), ('great_pyrenees', 200), ('american_bulldog', 200), ('pomeranian', 200), ('american_pit_bull_terrier', 200), ('yorkshire_terrier', 200), ('japanese_chin', 200), ('miniature_pinscher', 200), ('basset_hound', 200), ('saint_bernard', 200)]


## 자 이제 딥러닝을 내가 만들어보자
- 선생님이 없이 혼자하려니까 정말 어렵다 일단 해보자 
- import tensorflow as tf가 뭔데?
- tensorflow: 구글이 만든 머신러닝/딥러닝 라이브러리.
- 걍 내가 tensorflow 만든다 구글미안하다 쑤아가 간다 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
- ssuatensorflow gogo

### 역전파란? 멀까?

- 결과(손실)가 조금 바뀔 때, 각 입력/가중치가 얼마나 ‘책임’이 있는지 거꾸로 계산해 주는 절차
- 수학 규칙이 연쇄법칙(Chain Rule).
- 여기서 말하는 손실은 (실제값 - 예측값)^ 을 의미함 (=MSE)
- mse를 줄이기 위해서는 기울기를 줄여야 하니까 그래서 미분이 되어야 함
- 미분이 되어야 되기 때문에 연쇄법칙이 이용됨
- 연쇄법칙이 이용되는 이유는 모델의 손실 𝐿, L은 보통 여러 연산이 겹겹이 합성된 함수
- 이때 dy/dx는 바로 못 구하니까 조각조각 내서 곱해준다고 생각하면 됨
- 그래서 결론은 역전파 = 연쇄법칙 으로 보면 돼

In [39]:

from pathlib import Path
code = r'''
# ssuatensortflow/tensor.py
# class Tensor : 자동미분 코어
# ------------------------------------------------------------
# "값(data) + 그라디언트(grad) + 연산그래프(parent 연결) + 역전파 규칙(_backward)"
# 을 묶어서 다루는 아주 작은 자동미분 엔진의 코어입니다.
# ------------------------------------------------------------
import numpy as np

class Tensor:
    """
    숫자 배열(np.ndarray)을 감싸는 클래스.
    - data: 실제 값 (float32 배열)
    - requires_grad: 이 텐서에 대해 grad(∂L/∂self)를 추적할지 여부
    - grad: backward() 후 채워지는 그라디언트 버퍼 (data와 같은 shape)
    - _prev: 이 텐서를 만든 부모 텐서들(연산 그래프 연결용)
    - _op: 어떤 연산으로 만들어졌는지(디버깅/시각화용 라벨)
    - _backward: 이 노드에서 부모로 grad를 보내는 함수(연산별 규칙 저장)
    """
    def __init__(self, data, requires_grad=False, _children=(), _op=""):
        # 1. 값을 float32로 통일(연산 일관성/속도)
        self.data = np.array(data, dtype=np.float32)

        # 2. 이 텐서에 대해 미분을 추적할지
        self.requires_grad = requires_grad

        # 3. grad 버퍼: 추적할 때만 0으로 초기화, 아니면 None
        self.grad = np.zeros_like(self.data) if requires_grad else None

        # 4. 부모 연결: 연산이 결과 텐서를 만들 때 (부모들)을 넘겨줌
        self._prev = set(_children)

        # 5. 어떤 연산의 결과인지 라벨(디버깅용)
        self._op = _op

        # 6. 역전파 규칙 슬롯: 각 연산이 out을 만들 때 여기에 규칙을 주입
        self._backward = lambda: None

    @staticmethod
    def _to_tensor(x):
        """스칼라/ndarray도 Tensor로 통일해서 연산을 단순화."""
        return x if isinstance(x, Tensor) else Tensor(x)

    # -------------------- 기본 연산들 --------------------
    # +
    def __add__(self, other):
        """
        덧셈: z = x + y
        grad 규칙: ∂z/∂x = 1, ∂z/∂y = 1  → 부모의 grad에 out.grad를 그대로 더함
        """
        other = Tensor._to_tensor(other)
        out = Tensor(self.data + other.data,
                     requires_grad=self.requires_grad or other.requires_grad,
                     _children=(self, other), _op="+")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad + out.grad
            if other.requires_grad:
                other.grad = other.grad + out.grad
        out._backward = _backward
        return out
    # 교환법칙임 그니까 반대로 들어오는거 자리 바꿔서 인식 한다고 
    #  3 + x ==> x + 3 요런 느낌쓰 ㅇㅋ?
    def __radd__(self, other):
        return self.__add__(other)

    def sum(self):
        """
        전부합: s = x.sum()  (스칼라)
        grad 규칙: ∂s/∂x = 1(같은 shape) → out.grad * 1 을 부모로 보냄
        """
        out = Tensor(self.data.sum(),
                     requires_grad=self.requires_grad,
                     _children=(self,), _op="sum")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad + np.ones_like(self.data) * out.grad
        out._backward = _backward
        return out
    # 부호반전: y = -x
    def __neg__(self):
        """
        grad 규칙: ∂y/∂x = -1 → 부모 grad에 -out.grad 더하기
        """
        out = Tensor(-self.data, requires_grad=self.requires_grad, _children=(self,), _op="neg")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad - out.grad
        out._backward = _backward
        return out
    # - 뺄셈: x - y = x + (-y)
    def __sub__(self, other):
        
        other = Tensor._to_tensor(other)
        return self.__add__(-other)
    # 자리 빠꾼 - 
    def __rsub__(self, other):
        other = Tensor._to_tensor(other)
        return other.__sub__(self)
    #  원소곱: z = x * y
    def __mul__(self, other):
        """
        grad 규칙: ∂z/∂x = y, ∂z/∂y = x → 부모 grad에 상대방 data를 곱해 누적
        """
        other = Tensor._to_tensor(other)
        out = Tensor(self.data * other.data,
                     requires_grad=self.requires_grad or other.requires_grad,
                     _children=(self, other), _op="*")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad + out.grad * other.data
            if other.requires_grad:
                other.grad = other.grad + out.grad * self.data
        out._backward = _backward
        return out
    # 자리 바꾼 곱곱
    def __rmul__(self, other):
        return self.__mul__(other)


    # 평균: m = x.mean() (스칼라)
    def mean(self):
        """
        grad 규칙: ∂m/∂x = 1/N → ones_like(x) * (out.grad/N)
        """
        n = self.data.size
        out = Tensor(self.data.mean(),
                     requires_grad=self.requires_grad,
                     _children=(self,), _op="mean")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad + (np.ones_like(self.data) * (out.grad / n))
        out._backward = _backward
        return out
    #  거듭제곱: y = x ** p  (p는 스칼라)
    def pow(self, p: float):
        """
        grad 규칙: ∂y/∂x = p * x^(p-1)
        """
        out = Tensor(self.data ** p,
                     requires_grad=self.requires_grad,
                     _children=(self,), _op=f"pow{p}")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad + out.grad * (p * (self.data ** (p - 1)))
        out._backward = _backward
        return out

    def __matmul__(self, other):
        """
        행렬곱: Z = A @ B  (단순화: 2D @ 2D만 지원)
        grad 규칙:
          dL/dA = dL/dZ @ B^T
          dL/dB = A^T @ dL/dZ
        """
        other = Tensor._to_tensor(other)
        A, B = self.data, other.data
        out = Tensor(A @ B,
                     requires_grad=self.requires_grad or other.requires_grad,
                     _children=(self, other), _op="@")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad + out.grad @ B.T
            if other.requires_grad:
                other.grad = other.grad + A.T @ out.grad
        out._backward = _backward
        return out
    # 왜 음수값은 뱉어내고 양수 값만 받을까
    # z > 0 : “그 특징이 있다고 판단” → 값을 그대로 다음 층에 전달
    # z ≤ 0 : “그 특징이 없다/약하다” → 0(침묵) 으로 보냄
    # 특징이 있냐 없냐를 보고 판단 하는 함수라서 그런거임 ㅇㅇ
    def relu(self):
        """
        ReLU: y = max(0, x)
        grad 규칙: ∂y/∂x = 1(x>0), else 0  → 마스크로 곱해 전달
        """
        mask = (self.data > 0).astype(np.float32)
        out = Tensor(self.data * mask,
                     requires_grad=self.requires_grad,
                     _children=(self,), _op="relu")
        def _backward():
            if self.requires_grad:
                self.grad = self.grad + out.grad * mask
        out._backward = _backward
        return out

    # -------------------- 역전파 엔진 --------------------
    # 연쇄법칙(Chain Rule)을 이용해, 현재 계산 그래프의 모든 텐서에 대한 기울기(gradient)를 자동으로 계산해서 Tensor.grad에 채워넣는 함수

    def backward(self):
        """
        loss(보통 스칼라)에서 시작해 부모 방향으로 grad를 전파.
        # 절차:
        # 1) 위상정렬
        # 2) 모든 grad 0으로 초기화
        # 3) 루트(self)의 grad = 1 설정 (dL/dL = 1)
        # 4) 리스트를 뒤집어 각 노드의 _backward() 실행 → 부모 grad 누적
        """
        # 1. 그래프 순서 만들기  위상정렬(DFS)
        #    자식이 뒤, 부모가 앞
        topo, visited = [], set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for p in v._prev:
                    build(p)
                topo.append(v)
        build(self)

        # 2. 전파는 누적(+=) 이므로, 이전 값이 남아있지 않게 0에서 시작.
        for v in topo:
            if v.requires_grad:
                if v.grad is None:
                    v.grad = np.zeros_like(v.data)
                else:
                    v.grad[...] = 0.0

        # (3) 루트 grad = 1 (loss에서 시작)
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        else:
            self.grad[...] = 1.0

        # (4) 자식 - >부모로 역전파
        for v in reversed(topo):
            v._backward()
'''
Path("ssuatensortflow/tensor.py").write_text(code, encoding="utf-8")
print("ssuatensortflow/tensor.py 갱신 완료")


ssuatensortflow/tensor.py 갱신 완료


In [ ]:

from pathlib import Path
code = r'''
# ssuatensortflow/nn.py
# 손실 함수
# ------------------------------------------------------------
# 분류용 손실: 소프트맥스 + 크로스엔트로피(수치안정형)
# 다중분류(multi-class classification) 문제에서 사용되는 손실(loss) 함수
# logits: (B, C) Tensor, y_int: (B,) int64 numpy array
# ------------------------------------------------------------
import numpy as np
from .tensor import Tensor

def softmax_cross_entropy(logits: Tensor, y_int: np.ndarray) -> Tensor:
    """
    안정형 softmax:
    # 약간 softmax = 값을 확률로 바꿔준다고 생각하면 됨 
    # 모든 클래스의 합은 1이다
    # croos_sentropy = 예측값과 실제 값과의 불일치 정도라고 보면 되는데 
    # 정답 클래스의 확률을 높이고, 오답 클래스의 확률을 낮추도록
      z = logits - max(logits, axis=1)
      probs = exp(z) / sum(exp(z), axis=1)

    교차엔트로피:
      L = - 평균( log( probs[range(B), y] ) )

    역전파 규칙(유도 결과):
      dL/dlogits = (probs - onehot(y)) / B
    """
    assert logits.data.ndim == 2, "logits must be (B, C)"
    B, C = logits.data.shape
    y_int = y_int.astype(np.int64)

    # (1) 안정형 softmax
    z = logits.data - logits.data.max(axis=1, keepdims=True)  # overflow 방지
    exp = np.exp(z)
    probs = exp / exp.sum(axis=1, keepdims=True)

    # (2) cross-entropy
    eps = 1e-12
    loss_val = -np.mean(np.log(probs[np.arange(B), y_int] + eps))

    # (3) 결과 텐서(스칼라)
    out = Tensor(loss_val, requires_grad=logits.requires_grad,
                 _children=(logits,), _op="softmax_ce")

    # (4) 역전파: dL/dlogits = (probs - onehot)/B
    y_onehot = np.eye(C, dtype=np.float32)[y_int]
    def _backward():
        if logits.requires_grad:
            grad_logits = (probs - y_onehot) / B
            logits.grad = logits.grad + grad_logits
    out._backward = _backward
    return out
'''
Path("ssuatensortflow/nn.py").write_text(code, encoding="utf-8")
print(" ssuatensortflow/nn.py 갱신 완료")


 ssuatensortflow/nn.py 갱신 완료


In [74]:
from pathlib import Path
# ssuatensortflow/layers.py
# ------------------------------------------------------------
# 간단 레이어/옵티마이저 (개선판):
#   - add_bias: (B,F) + (1,F) 전용 덧셈(브로드캐스팅 없이 역전파 안전)
#   - dropout(x,p,train): 함수형 드롭아웃 (inverted 방식)
#   - Linear(in,out, init=...): Xavier/He 초기화 선택
#   - SGD: momentum / weight_decay(L2) / grad clipping 지원
# ------------------------------------------------------------
import numpy as np
code = r'''
# Tensor 임포트 (노트북/스크립트 실행 맥락 모두 지원)
import numpy as np
from .tensor import Tensor


# ======= 유틸: fan_in/out & 초기화 =======

def _fan_in_out(shape):
    if len(shape) == 2:
        fan_in, fan_out = shape[0], shape[1]
    else:
        # Conv 등 고려할 때의 일반식(여긴 2D만 쓰지만 안전하게)
        fan_in = shape[0] if len(shape) > 0 else 1
        fan_out = shape[1] if len(shape) > 1 else 1
    return fan_in, fan_out

def _xavier_uniform(shape, rng):
    fan_in, fan_out = _fan_in_out(shape)
    limit = np.sqrt(6.0 / (fan_in + fan_out))
    return rng.uniform(-limit, limit, size=shape).astype(np.float32)

def _he_uniform(shape, rng):
    fan_in, _ = _fan_in_out(shape)
    limit = np.sqrt(6.0 / max(1, fan_in))
    return rng.uniform(-limit, limit, size=shape).astype(np.float32)


# ======= Op: add_bias =======

def add_bias(z: Tensor, b: Tensor) -> Tensor:
    """
    (B,F) + (1,F) 형태의 편향 덧셈을 전용 op로 구현.
    역전파 시 b.grad는 배치 축으로 합산해 (1,F) 모양으로 누적.
    """
    out = Tensor(z.data + b.data,
                 requires_grad=z.requires_grad or b.requires_grad,
                 _children=(z, b), _op="add_bias")
    def _backward():
        if z.requires_grad:
            if z.grad is None:
                z.grad = np.zeros_like(z.data, dtype=z.data.dtype)
            z.grad = z.grad + out.grad
        if (b is not None) and b.requires_grad:
            if b.grad is None:
                b.grad = np.zeros_like(b.data, dtype=b.data.dtype)
            b.grad = b.grad + out.grad.sum(axis=0, keepdims=True)
    out._backward = _backward
    return out


# ======= Op: dropout (inverted) =======

def dropout(x: Tensor, p: float = 0.5, train: bool = True) -> Tensor:
    """
    함수형 드롭아웃 (학습시만 적용).
    inverted dropout: 학습 시 1/(1-p)로 스케일 → 추론 시 별도 보정 불필요.
    """
    assert 0.0 <= p < 1.0, "dropout p must be in [0,1)"
    if (not train) or p == 0.0:
        # 추론 모드 또는 p=0: 그래프는 유지
        out = Tensor(x.data.copy(), requires_grad=x.requires_grad, _children=(x,), _op="dropout_pass")
        def _backward():
            if x.requires_grad:
                if x.grad is None:
                    x.grad = np.zeros_like(x.data, dtype=x.data.dtype)
                x.grad = x.grad + out.grad
        out._backward = _backward
        return out

    keep = 1.0 - p
    rng = np.random.default_rng()
    mask = (rng.random(x.data.shape) < keep).astype(x.data.dtype)
    y_data = (x.data * mask) / keep

    out = Tensor(y_data, requires_grad=x.requires_grad, _children=(x,), _op="dropout")
    def _backward():
        if x.requires_grad:
            if x.grad is None:
                x.grad = np.zeros_like(x.data, dtype=x.data.dtype)
            x.grad = x.grad + (out.grad * mask) / keep
    out._backward = _backward
    return out


# ======= Layer: Linear =======

class Linear:
    """
    완전연결층: y = x @ W (+ b)
      - W: (in_features, out_features)
      - b: (1, out_features)
    init:
      - 'xavier' (기본)
      - 'he'     (ReLU 계열에서 권장)
    """
    def __init__(self, in_features: int, out_features: int,
                 bias: bool = True, rng=None, init: str = "xavier"):
        rng = np.random.default_rng() if rng is None else rng
        shape = (in_features, out_features)
        init = (init or "xavier").lower()

        if init == "he":
            W_init = _he_uniform(shape, rng)
        else:
            W_init = _xavier_uniform(shape, rng)

        self.W = Tensor(W_init, requires_grad=True)
        self.b = Tensor(np.zeros((1, out_features), np.float32), requires_grad=True) if bias else None

    def parameters(self):
        """학습 가능한 파라미터 리스트(W, b)."""
        return [p for p in (self.W, self.b) if p is not None]

    def __call__(self, x: Tensor) -> Tensor:
        """순전파: y = x @ W (+ b)"""
        y = x @ self.W
        if self.b is not None:
            y = add_bias(y, self.b)
        return y


# ======= Optimizer: SGD (momentum / L2 / grad clip) =======

class SGD:
    """
    확장 SGD:
      - momentum: v = m*v - lr*g ; p += v
      - weight_decay: g <- g + wd * p.data (L2 regularization)
      - max_norm: 전체 grad L2를 max_norm으로 클립
    """
    def __init__(self, params, lr=0.1, momentum=0.0, weight_decay=0.0, max_norm=None):
        self.params = list(params)
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.max_norm = max_norm
        self._vel = {id(p): np.zeros_like(p.data, dtype=p.data.dtype) for p in self.params}

    def _apply_weight_decay(self, p, g):
        if self.weight_decay != 0.0:
            g = g + self.weight_decay * p.data
        return g

    def _clip_grad_norm_(self):
        if self.max_norm is None:
            return
        total_sq = 0.0
        for p in self.params:
            if p.grad is not None:
                total_sq += float(np.sum(p.grad * p.grad))
        total_norm = np.sqrt(total_sq)
        if total_norm > self.max_norm and total_norm > 0:
            scale = self.max_norm / (total_norm + 1e-12)
            for p in self.params:
                if p.grad is not None:
                    p.grad *= scale

    def step(self):
        # 그라드 클리핑(옵션)
        self._clip_grad_norm_()

        for p in self.params:
            if p.grad is None:
                continue
            g = self._apply_weight_decay(p, p.grad)

            if self.momentum != 0.0:
                v = self._vel[id(p)]
                v[:] = self.momentum * v - self.lr * g
                p.data += v
            else:
                p.data -= self.lr * g

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad[...] = 0.0
'''

# importlib 쓰는 이유 
- 노트북에서 우리가 ssuatensortflow/tensor.py 파일을 고쳐도 이해를 못 할 수가 있으니까 리로드 할려고 쓰임 

In [ ]:
import importlib, ssuatensortflow as stf
# 함수 바뀐거 있으면 로드해라
importlib.reload(stf ) # 내가 만들어 둔 stf 약어 ㅇㅇ 


x = stf.Tensor([1,2,3], requires_grad=True)
y = stf.Tensor([10,20,30], requires_grad=True)

z = x + y      # [11, 22, 33]
s = z.sum()    # 66 (스칼라)
s.backward()

print("z.data:", z.data)   # [11. 22. 33.]
print("s.data:", s.data)   # 66.0
print("x.grad:", x.grad)   # [1. 1. 1.]
print("y.grad:", y.grad)   # [1. 1. 1.]


z.data: [11. 22. 33.]
s.data: 66.0
x.grad: [1. 1. 1.]
y.grad: [1. 1. 1.]


파이썬은 x + y를 내부적으로 x.__add__(y) 로 바꿔 호출

우리 Tensor.__add__ 메서드 안에서 직접 새 텐서를 만들 때 _op="+"를 넣어줬어.

그래서 결과 텐서의 _op가 "+"가 됨.

In [19]:
import importlib, ssuatensortflow as stf
importlib.reload(stf)

x = stf.Tensor([1,2,3], requires_grad=True)   # 리프: _op == ""
y = stf.Tensor([10,20,30], requires_grad=True)

z = x + y     # z._op == "+"
s = z.sum()   # s._op == "sum"

print("x._op:", x._op)  # ""
print("z._op:", z._op)  # "+"
print("s._op:", s._op)  # "sum"


x._op: 
z._op: +
s._op: sum


### 모듈 리로드

In [46]:
import importlib, numpy as np, pathlib, random
from PIL import Image
import ssuatensortflow as stf
import ssuatensortflow.nn as nn
import ssuatensortflow.layers as L
importlib.reload(stf); importlib.reload(nn); importlib.reload(L)

DATA_DIR = pathlib.Path("data")
assert DATA_DIR.exists(), DATA_DIR



In [47]:
import pathlib
DATA_DIR = pathlib.Path("data")
print("경로:", DATA_DIR.resolve())
print("exists?", DATA_DIR.exists())


경로: /Users/suakim/Documents/프로젝트/flutter 딥러닝 플젝/Model/data
exists? True


In [60]:
from collections import defaultdict
from PIL import Image
import numpy as np
import random

IMG_SIZE = 64  # 한 이미지를 64x64로 축소

# (1) 품종별 폴더 확인
breeds = sorted([p.name for p in DATA_DIR.iterdir() if p.is_dir()])
breed_to_idx = {b:i for i,b in enumerate(breeds)}
num_classes = len(breeds)
print(f"클래스 수: {num_classes}")
print("예시 품종 5개:", breeds[:5])

# (2) 이미지 경로 수집
files = []
for b in breeds:
    for jpg in (DATA_DIR/b).glob("*.jpg"):
        files.append((jpg, breed_to_idx[b]))
print(f"총 이미지 개수: {len(files)}")

# (3) train/val 8:2 분할
random.seed(0)
by_class = defaultdict(list)
for fp, y in files:
    by_class[y].append(fp)

train, val = [], []
for y, fps in by_class.items():
    random.shuffle(fps)
    k = int(len(fps)*0.8)
    train += [(fp, y) for fp in fps[:k]]
    val   += [(fp, y) for fp in fps[k:]]
print(f"Train: {len(train)}, Val: {len(val)}")

# (4) 이미지를 로드해 numpy로 변환하는 함수
# 흑백으로 바꾸는 거 
# def load_img_as_vec(path, size=IMG_SIZE):
#     im = Image.open(path).convert("L").resize((size, size))
#     arr = np.asarray(im, dtype=np.float32) / 255.0
#     return arr.reshape(-1)  # (D,)
#D = IMG_SIZE * IMG_SIZE

def load_img_as_vec(path, size=IMG_SIZE):
    im = Image.open(path).convert("RGB").resize((size, size))
    arr = np.asarray(im, dtype=np.float32) / 255.0
    return arr.reshape(-1)  # (D,)


print(f"입력 차원: {D}")


클래스 수: 25
예시 품종 5개: ['american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'boxer']
총 이미지 개수: 4978
Train: 3980, Val: 998
입력 차원: 4096


## 전체 이미지 인덱스 리스트 만들기 

In [49]:
def make_batches(pairs, batch_size=64, shuffle=True):
    idxs = np.arange(len(pairs))
    if shuffle:
        np.random.shuffle(idxs)
    for i in range(0, len(pairs), batch_size):
        j = idxs[i:i+batch_size]
        xs = [load_img_as_vec(pairs[k][0]) for k in j]
        ys = [pairs[k][1] for k in j]
        X = np.stack(xs).astype(np.float32)     
        y = np.array(ys, dtype=np.int64)        
        yield X, y


In [63]:
import importlib
import ssuatensortflow.tensor as T
import ssuatensortflow.layers as L
import ssuatensortflow.nn as nn
import ssuatensortflow as stf

# 1) 모듈을 'tensor -> layers -> nn -> stf' 순으로 리로드
importlib.reload(T)
importlib.reload(L)
importlib.reload(nn)
importlib.reload(stf)

# 2) (중요) 기존 모델/옵티마이저 객체 버리고 새로 생성
# 흑백일 때 차원이 적어서 
# D = IMG_SIZE * IMG_SIZE         # 이미 위에서 정의했음(64x64=4096)
# fc1 = L.Linear(D, 256, bias=True)
# fc2 = L.Linear(256, len(breeds), bias=True)

# 칼라로 바꾸고 차원이 늘었음 
D = IMG_SIZE * IMG_SIZE * 3  # 이제 3채널
fc1 = L.Linear(D, 1024)       # 차원 늘어나서 은닉층도 조금 크게!
fc2 = L.Linear(1024, num_classes)


def forward(X_np):
    x = stf.Tensor(X_np, requires_grad=False)  # 이제 '새 Tensor' 클래스
    h = fc1(x).relu()
    logits = fc2(h)
    return logits

opt = L.SGD(fc1.parameters() + fc2.parameters(), lr=0.2)


In [64]:
EPOCHS = 10
BATCH  = 64

for ep in range(1, EPOCHS+1):
    losses = []
    for X, y in make_batches(train, batch_size=BATCH, shuffle=True):
        logits = forward(X)
        loss   = nn.softmax_cross_entropy(logits, y)
        loss.backward()
        opt.step()
        losses.append(float(loss.data))
    print(f"[{ep:02d}] loss={np.mean(losses):.4f}")


[01] loss=4.6506
[02] loss=3.1939
[03] loss=3.2089
[04] loss=3.1773
[05] loss=3.1771
[06] loss=3.1775
[07] loss=3.2173
[08] loss=3.1760
[09] loss=3.1726
[10] loss=3.2023


In [75]:
import importlib
import ssuatensortflow.tensor as T
import ssuatensortflow.layers as L
import ssuatensortflow.nn as nn
import ssuatensortflow as stf

# 1) 모듈을 'tensor -> layers -> nn -> stf' 순으로 리로드
importlib.reload(T)
importlib.reload(L)
importlib.reload(nn)
importlib.reload(stf)

# D = IMG_SIZE * IMG_SIZE * 3
# num_classes = len(breeds)

fc1 = L.Linear(D, 1024, init="he")
fc2 = L.Linear(1024, num_classes, init="xavier")

opt = L.SGD(fc1.parameters() + fc2.parameters(),
            lr=0.01, momentum=0.9, weight_decay=1e-4, max_norm=5.0)

def forward(X_np, train=True):
    x = stf.Tensor(X_np, requires_grad=False)
    h = fc1(x).relu()
    h = L.dropout(h, p=0.3, train=train)  # 학습 시만 활성
    logits = fc2(h)
    return logits


In [172]:
import numpy as np
import ssuatensortflow as stf
import ssuatensortflow.layers as L
import ssuatensortflow.nn as nn

D, num_classes = 64*64*3, 10  # 예시용
X_dummy = np.random.rand(8, D).astype(np.float32)
y_dummy = np.random.randint(0, num_classes, size=(8,))

fc1 = L.Linear(D, 1024, init="he")
fc2 = L.Linear(1024, num_classes, init="xavier")

opt = L.SGD(fc1.parameters() + fc2.parameters(),
            lr=0.01, momentum=0.9, weight_decay=1e-4, max_norm=5.0)

def forward(X_np, train=True):
    x = stf.Tensor(X_np, requires_grad=False)
    h = fc1(x).relu()
    h = L.dropout(h, p=0.3, train=train)
    logits = fc2(h)
    return logits

# 순전파 + 역전파 테스트
logits = forward(X_dummy)
loss = nn.softmax_cross_entropy(logits, y_dummy)
opt.zero_grad()
loss.backward()
opt.step()
print("✅ 테스트 성공, loss =", float(loss.data))


✅ 테스트 성공, loss = 2.048614501953125


In [80]:
# 오버핏 테스트 전용
opt = L.SGD(fc1.parameters() + fc2.parameters(), lr=0.05)  # momentum=0, wd=0
def forward_overfit(X_np):
    x = stf.Tensor(X_np, requires_grad=False)
    h = fc1(x).relu()
    logits = fc2(h)  # dropout 끔
    return logits


In [173]:
# train_rgb_mlp.py  (1) 데이터 준비
from pathlib import Path
from collections import defaultdict
from PIL import Image
import numpy as np, random

DATA_DIR = Path("data")          # 품종 폴더들이 들어있는 루트
IMG_SIZE = 64
random.seed(0); np.random.seed(0)

# 1) 클래스/라벨
breeds = sorted([p.name for p in DATA_DIR.iterdir() if p.is_dir()])
breed_to_idx = {b:i for i,b in enumerate(breeds)}
num_classes = len(breeds)

files = []
for b in breeds:
    for jpg in (DATA_DIR/b).glob("*.jpg"):
        files.append((jpg, breed_to_idx[b]))

# 2) 클래스별 8:2 분할
by_class = defaultdict(list)
for fp, y in files: by_class[y].append(fp)

train, val = [], []
for y, fps in by_class.items():
    random.shuffle(fps)
    k = int(len(fps)*0.8)
    train += [(fp, y) for fp in fps[:k]]
    val   += [(fp, y) for fp in fps[k:]]

# 3) 채널별 mean/std 추정
def compute_mean_std(pairs, n_samples=1024):
    sel = pairs if len(pairs)<=n_samples else random.sample(pairs, n_samples)
    xs = []
    for fp,_ in sel:
        im = Image.open(fp).convert("RGB").resize((IMG_SIZE, IMG_SIZE))
        arr = np.asarray(im, np.float32)/255.0   # (H,W,3)
        xs.append(arr.reshape(-1,3))
    X = np.concatenate(xs, axis=0)
    mean = X.mean(axis=0); std = X.std(axis=0) + 1e-6
    return mean, std

mean3, std3 = compute_mean_std(train)

# 4) 로더 & 미니배치
def load_img_as_vec(path):
    im = Image.open(path).convert("RGB").resize((IMG_SIZE, IMG_SIZE), resample=Image.BILINEAR)
    arr = np.asarray(im, np.float32)/255.0
    arr = (arr - mean3)/std3
    return arr.reshape(-1)  # (D,)

def iterate_minibatches(pairs, batch_size=64, shuffle=True):
    idx = np.arange(len(pairs))
    if shuffle: np.random.shuffle(idx)
    for s in range(0, len(pairs), batch_size):
        sel = idx[s:s+batch_size]
        Xb = np.stack([load_img_as_vec(pairs[i][0]) for i in sel], axis=0)
        yb = np.array([pairs[i][1] for i in sel], dtype=np.int64)
        yield Xb, yb

D = IMG_SIZE*IMG_SIZE*3
print(f"[DATA] classes={num_classes}, train={len(train)}, val={len(val)}, D={D}, mean={mean3}, std={std3}")


[DATA] classes=25, train=3980, val=998, D=12288, mean=[0.48045987 0.45337072 0.39187315], std=[0.2551374  0.25123423 0.2606304 ]


In [174]:
# train_rgb_mlp.py  (2) 모델/옵티마이저/forward
import importlib
import ssuatensortflow as stf
import ssuatensortflow.layers as L
import ssuatensortflow.nn as nn

importlib.reload(L); importlib.reload(nn)

fc1 = L.Linear(D, 1024, init="he")
fc2 = L.Linear(1024, num_classes, init="xavier")

opt = L.SGD(fc1.parameters() + fc2.parameters(),
            lr=0.01, momentum=0.9, weight_decay=1e-4, max_norm=5.0)

def forward(X_np, train=True):
    x = stf.Tensor(X_np, requires_grad=False)
    h = fc1(x).relu()
    h = L.dropout(h, p=0.3, train=train)   # 학습 때만 활성
    logits = fc2(h)
    return logits


In [175]:
# train_rgb_mlp.py  (3) 평가 + 학습 루프
def evaluate(pairs, batch_size=128):
    losses, correct, total = [], 0, 0
    for Xb, yb in iterate_minibatches(pairs, batch_size=batch_size, shuffle=False):
        logits = forward(Xb, train=False)
        loss = nn.softmax_cross_entropy(logits, yb)
        losses.append(float(loss.data))
        pred = np.argmax(logits.data, axis=1)
        correct += int((pred == yb).sum())
        total   += int(yb.size)
    return (np.mean(losses) if losses else float("nan")), (correct/total if total else 0.0)

EPOCHS = 10
best_val = float("inf")
patience, bad = 3, 0

for ep in range(1, EPOCHS+1):
    # ---- Train ----
    train_losses = []
    train_correct, train_total = 0, 0

    for Xb, yb in iterate_minibatches(train, batch_size=64, shuffle=True):
        logits = forward(Xb, train=True)
        loss = nn.softmax_cross_entropy(logits, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()

        train_losses.append(float(loss.data))
        pred = np.argmax(logits.data, axis=1)
        train_correct += int((pred == yb).sum())
        train_total   += int(yb.size)

    tr_loss = np.mean(train_losses)
    tr_acc  = train_correct / train_total

    # ---- Val ----
    val_loss, val_acc = evaluate(val)
    print(f"[{ep:02d}] train_loss={tr_loss:.4f} acc={tr_acc:.3f} | "
          f"val_loss={val_loss:.4f} acc={val_acc:.3f} | lr={opt.lr:g}")

    # plateau → lr 반감
    if val_loss < best_val - 1e-4:
        best_val = val_loss
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            opt.lr *= 0.5
            bad = 0
            print(f"  ↳ plateau detected: reduce lr to {opt.lr:g}")


[01] train_loss=4.1572 acc=0.093 | val_loss=3.8882 acc=0.122 | lr=0.01
[02] train_loss=3.2960 acc=0.220 | val_loss=3.9292 acc=0.118 | lr=0.01
[03] train_loss=2.6762 acc=0.312 | val_loss=4.0136 acc=0.142 | lr=0.01
[04] train_loss=2.3449 acc=0.395 | val_loss=3.8690 acc=0.138 | lr=0.01
[05] train_loss=1.9013 acc=0.470 | val_loss=4.1558 acc=0.144 | lr=0.01
[06] train_loss=1.6769 acc=0.535 | val_loss=4.4493 acc=0.147 | lr=0.01
[07] train_loss=1.4803 acc=0.591 | val_loss=4.4037 acc=0.152 | lr=0.01
  ↳ plateau detected: reduce lr to 0.005
[08] train_loss=0.9989 acc=0.712 | val_loss=4.0707 acc=0.156 | lr=0.005
[09] train_loss=0.7182 acc=0.787 | val_loss=4.2060 acc=0.171 | lr=0.005
[10] train_loss=0.6224 acc=0.831 | val_loss=4.2276 acc=0.161 | lr=0.005
  ↳ plateau detected: reduce lr to 0.0025


transforms: 데이터 다양화 → 과적합 감소, 일반화↑

BatchNorm: 내부 공변량 변화 감소 → 수렴 빨라짐 + 초기화/학습률에 덜 민감

ReduceLROnPlateau: 멈추면 자동으로 lr 낮춤 → 더 낮은 최소점 탐색

L2/Dropout 강화: 학습셋 암기(오버핏) 억제 → val 성능 개선

Tensor 연산 구조

Linear, BatchNorm1d, SGD 같은 기본 레이어와 옵티마이저

softmax_cross_entropy 손실

DataLoader, Compose 등 데이터 파이프라인

In [177]:
import ssuatensortflow.transforms as Tfm
import ssuatensortflow.utils as U
import ssuatensortflow.schedulers as S
U.seed_everything(0)


In [178]:
train_tf = Tfm.Compose([
    Tfm.RandomHorizontalFlip(p=0.5),
    Tfm.RandomPadCrop(IMG_SIZE, pad=4),
    Tfm.ToRGBResize(IMG_SIZE),
    Tfm.Normalize01(mean3, std3)
])
val_tf = Tfm.Compose([
    Tfm.ToRGBResize(IMG_SIZE),
    Tfm.Normalize01(mean3, std3)
])

def load_img_as_vec(path, train_mode=False):
    img = Image.open(path)
    tfm = train_tf if train_mode else val_tf
    arr = tfm(img)                 # (H,W,3)
    return Tfm.to_vec(arr)         # (D,)


In [179]:
def iterate_minibatches(pairs, batch_size=64, shuffle=True, train_mode=False):
    idx = np.arange(len(pairs))
    if shuffle: np.random.shuffle(idx)
    for s in range(0, len(pairs), batch_size):
        sel = idx[s:s+batch_size]
        Xb = np.stack([load_img_as_vec(pairs[i][0], train_mode=train_mode) for i in sel], axis=0)
        yb = np.array([pairs[i][1] for i in sel], dtype=np.int64)
        yield Xb, yb


In [183]:
fc1 = L.Linear(D, 512, init="he")
bn1 = L.BatchNorm1d(512)          # ★ 추가
fc2 = L.Linear(512, num_classes, init="xavier")

opt = L.SGD(fc1.parameters() + bn1.parameters() + fc2.parameters(),
            lr=0.005, momentum=0.9, weight_decay=3e-4, max_norm=5.0)

def forward(X_np, train=True):
    x = stf.Tensor(X_np, requires_grad=False)
    h = fc1(x)
    h = bn1(h, train=train)       # ★ BN
    h = h.relu()
    h = L.dropout(h, p=0.5, train=train)
    logits = fc2(h)
    return logits


In [184]:
scheduler = S.ReduceLROnPlateau(opt, factor=0.5, patience=2, min_lr=1e-6, threshold=1e-4)
best_path = "best_fc_bn.npz"
best_val = float("inf")


In [185]:
# 저장
if val_loss < best_val - 1e-4:
    best_val = val_loss
    U.save_checkpoint(best_path, [fc1, fc2])  # bn1은 러닝통계만 있고 가중치 gamma/beta는 bn1.parameters()로 포함(이미 opt에 포함)
# 스케줄
reduced = scheduler.step(val_loss)
if reduced:
    print(f"  ↳ ReduceLROnPlateau: lr -> {opt.lr:g}")


In [186]:
import importlib, inspect
import ssuatensortflow.layers as L
importlib.reload(L)
print("BatchNorm1d 존재? ->", hasattr(L, "BatchNorm1d"))
print("layers.py 경로:", inspect.getfile(L))


BatchNorm1d 존재? -> True
layers.py 경로: /Users/suakim/Documents/프로젝트/flutter 딥러닝 플젝/Model/ssuatensortflow/layers.py


In [188]:
for Xb, yb in iterate_minibatches(train, batch_size=64, shuffle=True, train_mode=True):
    ...
def evaluate(pairs, batch_size=128):
    losses, correct, total = [], 0, 0
    for Xb, yb in iterate_minibatches(pairs, batch_size=batch_size, shuffle=False, train_mode=False):
        logits = forward(Xb, train=False)
        loss = nn.softmax_cross_entropy(logits, yb)
        losses.append(float(loss.data))
        pred = np.argmax(logits.data, axis=1)
        correct += int((pred == yb).sum()); total += int(yb.size)
    return (np.mean(losses) if losses else float("nan")), (correct/total if total else 0.0)

EPOCHS = 10
best_path = "best_fc_bn.npz"
best_val = float("inf")

for ep in range(1, EPOCHS+1):
    # ---- Train (증강 ON) ----
    train_losses = []; train_correct = train_total = 0
    for Xb, yb in iterate_minibatches(train, batch_size=64, shuffle=True, train_mode=True):
        logits = forward(Xb, train=True)
        loss = nn.softmax_cross_entropy(logits, yb)
        opt.zero_grad(); loss.backward(); opt.step()

        train_losses.append(float(loss.data))
        pred = np.argmax(logits.data, axis=1)
        train_correct += int((pred == yb).sum()); train_total += int(yb.size)

    tr_loss = float(np.mean(train_losses)); tr_acc = train_correct/train_total

    # ---- Val ----
    val_loss, val_acc = evaluate(val)

    print(f"[{ep:02d}] train_loss={tr_loss:.4f} acc={tr_acc:.3f} | "
          f"val_loss={val_loss:.4f} acc={val_acc:.3f} | lr={opt.lr:g}")

    # ---- 저장 ----
    if val_loss < best_val - 1e-4:
        best_val = val_loss
        U.save_checkpoint(best_path, [fc1, fc2])   # bn1의 gamma/beta는 opt에 이미 포함되어 학습됨

    # ---- 스케줄 ----
    reduced = scheduler.step(val_loss)
    if reduced:
        print(f"  ↳ ReduceLROnPlateau: lr -> {opt.lr:g}")



[01] train_loss=3.7451 acc=0.048 | val_loss=3.2558 acc=0.054 | lr=0.005
[02] train_loss=3.5702 acc=0.058 | val_loss=3.2366 acc=0.044 | lr=0.005
[03] train_loss=3.5212 acc=0.052 | val_loss=3.2271 acc=0.053 | lr=0.005
[04] train_loss=3.4932 acc=0.065 | val_loss=3.2243 acc=0.049 | lr=0.005
[05] train_loss=3.4488 acc=0.059 | val_loss=3.2216 acc=0.061 | lr=0.005
[06] train_loss=3.4062 acc=0.059 | val_loss=3.2326 acc=0.051 | lr=0.005
[07] train_loss=3.3662 acc=0.064 | val_loss=3.2158 acc=0.056 | lr=0.005
[08] train_loss=3.3643 acc=0.066 | val_loss=3.2213 acc=0.056 | lr=0.005
[09] train_loss=3.3542 acc=0.067 | val_loss=3.2205 acc=0.047 | lr=0.005
  ↳ ReduceLROnPlateau: lr -> 0.0025
[10] train_loss=3.2980 acc=0.069 | val_loss=3.2130 acc=0.057 | lr=0.0025


In [189]:
np.savez("prep.npz", mean3=mean3, std3=std3, breeds=np.array(breeds, dtype=object))
print("저장:", "prep.npz", best_path)


저장: prep.npz best_fc_bn.npz


In [190]:
import numpy as np
from ssuatensortflow.tensor import Tensor
from ssuatensortflow.layers import Linear, SGD
from ssuatensortflow.nn import softmax_cross_entropy
from ssuatensortflow.utils import seed_everything
from ssuatensortflow.transforms import to_vec

seed_everything(0)

# 가짜 데이터 (B=16, C=10), 입력 32x32x3
B, C = 16, 10
X_np = np.random.randn(B, 32*32*3).astype(np.float32) * 0.2
y_np = np.random.randint(0, C, size=(B,), dtype=np.int64)

# 아주 작은 MLP (BN 빼고 먼저 확인)
l1 = Linear(32*32*3, 128, init="he")
l2 = Linear(128, C, init="xavier")
layers = [l1, l2]

opt = SGD([p for L in layers for p in L.parameters()], lr=1e-2, momentum=0.9, weight_decay=0.0)

for t in range(1000):
    x = Tensor(X_np, requires_grad=False)           # (B, D)
    h = l1(x).relu()
    logits = l2(h)
    loss = softmax_cross_entropy(logits, y_np)

    opt.zero_grad()
    loss.backward()
    opt.step()

    if t % 100 == 0:
        pred = np.argmax(logits.data, axis=1)
        acc = (pred == y_np).mean()
        print(f"[{t}] loss={loss.data:.4f} acc={acc:.3f}")

# 기대: acc가 0.9~1.0 가까이 올라감(과적합)


[0] loss=2.4066 acc=0.125
[100] loss=0.0056 acc=1.000
[200] loss=0.0033 acc=1.000
[300] loss=0.0023 acc=1.000
[400] loss=0.0018 acc=1.000
[500] loss=0.0014 acc=1.000
[600] loss=0.0012 acc=1.000
[700] loss=0.0010 acc=1.000
[800] loss=0.0009 acc=1.000
[900] loss=0.0008 acc=1.000


In [198]:

import importlib, sys, os
sys.path.append(os.path.abspath("."))
importlib.invalidate_caches()
import ssuatensortflow
importlib.reload(ssuatensortflow)
from ssuatensortflow.data import build_index, split_train_val, ImageFolderDataset, DataLoader



In [199]:
import importlib, sys, os
sys.path.append(os.path.abspath("."))
importlib.invalidate_caches()
import ssuatensortflow
importlib.reload(ssuatensortflow)

from ssuatensortflow.data import build_index, split_train_val, ImageFolderDataset, DataLoader


## 데이터 로드 + DataLoader 준비

In [202]:
from ssuatensortflow.transforms import Compose, ToRGBResize, RandomHorizontalFlip, RandomPadCrop, Normalize01, to_chw
from ssuatensortflow.utils import seed_everything
import numpy as np

seed_everything(0)

data_root = "data"

paths, labels, classes = build_index(data_root)
print("클래스 수:", len(classes))
print("예시 클래스 5개:", classes[:5])

(train_paths, train_labels), (val_paths, val_labels) = split_train_val(paths, labels, val_ratio=0.1, seed=0)
print("train:", len(train_paths), " | val:", len(val_paths))

# 변환 정의
train_tf = Compose([
    ToRGBResize(64),
    RandomPadCrop(64, pad=4),
    RandomHorizontalFlip(0.5),
    Normalize01([0.5,0.5,0.5],[0.5,0.5,0.5]),
])
val_tf = Compose([
    ToRGBResize(64),
    Normalize01([0.5,0.5,0.5],[0.5,0.5,0.5]),
])

train_ds = ImageFolderDataset(train_paths, train_labels, transform=train_tf)
val_ds   = ImageFolderDataset(val_paths, val_labels, transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True,  drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, drop_last=False)


클래스 수: 25
예시 클래스 5개: ['american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'boxer']
train: 4481  | val: 497


## 모델 구성 + 학습 루프 실행

In [234]:
import os

DATA_ROOT = "data"
print("DATA_ROOT:", DATA_ROOT)
print("하위 폴더 목록:", os.listdir(DATA_ROOT))


DATA_ROOT: data
하위 폴더 목록: ['wheaten_terrier', 'great_pyrenees', 'american_bulldog', 'pomeranian', 'american_pit_bull_terrier', 'yorkshire_terrier', 'japanese_chin', 'english_cocker_spaniel', 'miniature_pinscher', 'basset_hound', 'saint_bernard', 'chihuahua', 'newfoundland', 'pug', 'havanese', 'beagle', 'german_shorthaired', 'staffordshire_bull_terrier', 'samoyed', 'scottish_terrier', 'leonberger', 'keeshond', 'boxer', 'english_setter', 'shiba_inu']


In [6]:
# train_ssua_standalone.py (v2: 과적합 완화 튜닝 반영)
# ------------------------------------------------------------
# - transforms / dataset / dataloader / split 내장
# - ssuatensortflow: Tensor / Linear / BatchNorm1d / SGD / dropout / LS-CE / EarlyStopping 사용
# - 변경점: 입력/히든 축소, weight_decay↑, label smoothing↑, dropout↑, PadCrop 증강, 간단 LR decay
# - DataLoader에서 (B,H,W,C) → (B,F) 평탄화
# ------------------------------------------------------------
import os, sys, time, random, glob
import numpy as np

# 0) ssuatensortflow 경로 추가
sys.path.insert(0, os.path.join(os.getcwd(), "Model"))

# 1) 라이브러리 임포트 (공주님 라이브러리)
from ssuatensortflow.tensor import Tensor
from ssuatensortflow.layers import Linear, BatchNorm1d, SGD, dropout  # 함수형 dropout
from ssuatensortflow.nn import cross_entropy_with_label_smoothing
from ssuatensortflow.callbacks import EarlyStopping

# ============================================================
# A. 이 파일 안에서만 쓰는 전처리/데이터 유틸 (버전차이 방지용)
# ============================================================
from PIL import Image

class Compose:
    def __init__(self, transforms): self.transforms = transforms
    def __call__(self, img):
        for t in self.transforms: img = t(img)
        return img

class ToRGBResize:
    def __init__(self, width, height):
        self.w, self.h = width, height
    def __call__(self, img):
        # img 타입에 상관없이 안전하게 PIL.Image로 변환
        if isinstance(img, str):
            img = Image.open(img)
        elif isinstance(img, np.ndarray):
            arr = img
            if arr.dtype != np.uint8:
                arr = np.clip(arr * (255.0 if arr.max() <= 1.0 else 1.0), 0, 255).astype(np.uint8)
            img = Image.fromarray(arr)
        img = img.convert("RGB").resize((self.w, self.h))
        return np.array(img, dtype=np.float32) / 255.0  # 0~1 스케일

class Normalize01:
    def __init__(self, mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)):
        self.mean = np.array(mean, dtype=np.float32)
        self.std  = np.array(std,  dtype=np.float32)
    def __call__(self, img):
        x = img.data if isinstance(img, Tensor) else img
        return (x - self.mean) / self.std

class RandomHorizontalFlip:
    def __init__(self, p=0.5): self.p = p
    def __call__(self, img):
        if isinstance(img, str):
            return img  # 경로면 다음 변환에서 처리
        if np.random.rand() < self.p:
            if isinstance(img, np.ndarray):
                return np.fliplr(img)
        return img

class RandomPadCrop:
    """이미지 가장자리를 반사 패딩 후 원래 크기로 랜덤 크롭"""
    def __init__(self, padding=8):
        self.padding = padding
    def __call__(self, img):
        if isinstance(img, str):
            return img  # 경로 문자열은 다음 변환에서 처리
        if self.padding <= 0:
            return img
        h, w, c = img.shape
        pad = self.padding
        padded = np.pad(img, ((pad,pad),(pad,pad),(0,0)), mode="reflect")
        top  = np.random.randint(0, 2*pad)
        left = np.random.randint(0, 2*pad)
        return padded[top:top+h, left:left+w]

def stratified_split(data_root, val_ratio=0.15, seed=42):
    """data_root/{class}/*.jpg 구조를 클래스 비율 유지하며 분할"""
    rng = random.Random(seed)
    classes = sorted([d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))])
    cls_to_idx = {c:i for i,c in enumerate(classes)}
    train, val = [], []
    for c in classes:
        files = []
        for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp"):
            files += glob.glob(os.path.join(data_root, c, ext))
        files.sort(); rng.shuffle(files)
        k = int(len(files) * (1 - val_ratio))
        train += [(f, cls_to_idx[c]) for f in files[:k]]
        val   += [(f, cls_to_idx[c]) for f in files[k:]]
    rng.shuffle(train); rng.shuffle(val)
    return train, val

class ImageFolderDataset:
    """(path, class_idx) 리스트를 받아 이미지를 로드/전처리"""
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform
        self.classes = sorted(set([y for _, y in samples]))
        self.num_classes = len(self.classes)
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        path, y = self.samples[idx]
        if self.transform is not None:
            x = self.transform(path)  # 변환 파이프라인에 로드 포함
        else:
            img = Image.open(path).convert("RGB")
            x = np.array(img, dtype=np.float32) / 255.0
        if not isinstance(x, Tensor):
            x = Tensor(x.astype(np.float32))  # Tensor로 감싸기 (H,W,C)
        return x, np.int64(y)

class DataLoader:
    def __init__(self, dataset, batch_size=64, shuffle=True, drop_last=False):
        self.ds, self.bs, self.shuffle, self.drop_last = dataset, batch_size, shuffle, drop_last
    def __iter__(self):
        idxs = list(range(len(self.ds)))
        if self.shuffle: random.shuffle(idxs)
        batch = []
        for i in idxs:
            batch.append(self.ds[i])
            if len(batch) == self.bs:
                yield self._collate(batch); batch = []
        if batch and not self.drop_last:
            yield self._collate(batch)
    def __len__(self):
        n = len(self.ds) // self.bs
        if len(self.ds) % self.bs and not self.drop_last: n += 1
        return max(1, n)
    def _collate(self, batch):
        # (B,H,W,C) 스택 후 (B,F) 평탄화
        xs, ys = zip(*batch)
        x_np = np.stack([x.data for x in xs], axis=0).astype(np.float32)
        if x_np.ndim == 4:
            if x_np.shape[-1] == 3:  # (B,H,W,C)
                B = x_np.shape[0]
                x_np = x_np.reshape(B, -1)
            else:  # (B,3,H,W)
                B, C, H, W = x_np.shape
                x_np = x_np.reshape(B, C*H*W)
        return Tensor(x_np), np.array(ys, dtype=np.int64)

def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed)

def save_checkpoint(path, state: dict):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    np.savez_compressed(path, **state)

# ============================================================
# B. 설정 (v2 튜닝)
# ============================================================
# ★ 공주님 실제 데이터 경로 (캡처 기준 'Model/data')
DATA_ROOT = "data"

IMG_SIZE  = 160      # 224 -> 160
BATCH     = 64
EPOCHS    = 60
LR        = 5e-3     # 1e-2 -> 5e-3
MOM       = 0.9
WD        = 5e-4     # 1e-4 -> 5e-4
SMOOTH    = 0.2      # 0.1 -> 0.2
PATIENCE  = 12       # 8  -> 12
VAL_RATIO = 0.15
SEED      = 42

HIDDEN    = 256      # 512 -> 256

# ============================================================
# C. 데이터
# ============================================================
seed_everything(SEED)

def build_transforms(split):
    if split == "train":
        return Compose([
            ToRGBResize(IMG_SIZE, IMG_SIZE),
            RandomPadCrop(padding=8),     # ★ 추가 증강
            RandomHorizontalFlip(0.5),
            Normalize01(),
        ])
    else:
        return Compose([
            ToRGBResize(IMG_SIZE, IMG_SIZE),
            Normalize01(),
        ])

if not os.path.isdir(DATA_ROOT):
    raise FileNotFoundError(f"DATA_ROOT가 없습니다: {DATA_ROOT}")

train_files, val_files = stratified_split(DATA_ROOT, val_ratio=VAL_RATIO, seed=SEED)
train_ds = ImageFolderDataset(train_files, transform=build_transforms("train"))
val_ds   = ImageFolderDataset(val_files,   transform=build_transforms("val"))

num_classes = train_ds.num_classes
in_features = IMG_SIZE * IMG_SIZE * 3

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, drop_last=False)

# ============================================================
# D. 모델
# ============================================================
class FCNet:
    def __init__(self, in_features, hidden=HIDDEN, num_classes=10):
        self.fc1 = Linear(in_features, hidden, init="he")
        self.bn1 = BatchNorm1d(hidden)
        self.fc2 = Linear(hidden, num_classes, init="xavier")
        self.training = True

    def parameters(self):
        return self.fc1.parameters() + self.bn1.parameters() + self.fc2.parameters()

    def train(self): self.training = True
    def eval(self):  self.training = False

    def relu(self, x):
        mask = (x.data > 0).astype(x.data.dtype)
        return x * mask

    def forward(self, x):
        # DataLoader에서 이미 (B,F)
        y = self.fc1(x)
        y = self.bn1(y, train=self.training)
        y = self.relu(y)
        y = dropout(y, p=0.6, train=self.training)  # 0.4 -> 0.6
        y = self.fc2(y)
        return y

    # 얼리 스탑 저장/복원
    def state_dict(self):
        return {
            "fc1.W": self.fc1.W.data.copy(),
            "fc1.b": None if self.fc1.b is None else self.fc1.b.data.copy(),
            "bn1.gamma": self.bn1.gamma.data.copy(),
            "bn1.beta":  self.bn1.beta.data.copy(),
            "bn1.running_mean": self.bn1.running_mean.copy(),
            "bn1.running_var":  self.bn1.running_var.copy(),
            "fc2.W": self.fc2.W.data.copy(),
            "fc2.b": None if self.fc2.b is None else self.fc2.b.data.copy(),
        }
    def load_state_dict(self, sd):
        self.fc1.W.data[:] = sd["fc1.W"]
        if self.fc1.b is not None and sd["fc1.b"] is not None:
            self.fc1.b.data[:] = sd["fc1.b"]
        self.bn1.gamma.data[:] = sd["bn1.gamma"]
        self.bn1.beta.data[:]  = sd["bn1.beta"]
        self.bn1.running_mean[:] = sd["bn1.running_mean"]
        self.bn1.running_var[:]  = sd["bn1.running_var"]
        self.fc2.W.data[:] = sd["fc2.W"]
        if self.fc2.b is not None and sd["fc2.b"] is not None:
            self.fc2.b.data[:] = sd["fc2.b"]

model = FCNet(in_features, hidden=HIDDEN, num_classes=num_classes)
optimizer = SGD(model.parameters(), lr=LR, momentum=MOM, weight_decay=WD)
early = EarlyStopping(patience=PATIENCE)

# ============================================================
# E. 학습/평가 루프 (+ 간단 LR decay)
# ============================================================
def run_one_epoch(loader, train=True):
    if train: model.train()
    else:     model.eval()

    total_loss, total_correct, total_count = 0.0, 0, 0
    for x, y in loader:
        logits = model.forward(x)
        loss = cross_entropy_with_label_smoothing(logits, y, eps=SMOOTH)

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += float(loss.data)
        pred = np.argmax(logits.data, axis=1)
        total_correct += int((pred == y).sum())
        total_count  += y.shape[0]

    return total_loss / max(1, len(loader)), total_correct / max(1, total_count)

# ============================================================
# F. 실행
# ============================================================
best_path = os.path.join("Model", "checkpoints", "best_fc_bn.npz")
os.makedirs(os.path.dirname(best_path), exist_ok=True)

prev_val_loss = float("inf")

print(f"Classes: {num_classes}, Train: {len(train_ds)}, Val: {len(val_ds)}")
for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    tr_loss, tr_acc = run_one_epoch(train_loader, True)
    va_loss, va_acc = run_one_epoch(val_loader, False)

    # 간단한 plateau 감지 → lr 0.9배
    if epoch > 3 and abs(prev_val_loss - va_loss) < 1e-3:
        optimizer.lr *= 0.9
    prev_val_loss = va_loss

    early.update(va_loss, model.state_dict, model.load_state_dict)

    try: save_checkpoint(best_path, model.state_dict())
    except Exception: pass

    dt = time.time() - t0
    print(f"[{epoch:03d}] lr={optimizer.lr:.5f}  train loss={tr_loss:.4f} acc={tr_acc:.3f} | "
          f"val loss={va_loss:.4f} acc={va_acc:.3f}  ({dt:.1f}s)")

    if early.stopped:
        print(f"Early stopped at epoch {epoch}")
        break

print("Done.")


Classes: 25, Train: 4229, Val: 749
[001] lr=0.00500  train loss=3.6698 acc=0.075 | val loss=3.0866 acc=0.123  (28.0s)
[002] lr=0.00500  train loss=3.3722 acc=0.092 | val loss=3.0427 acc=0.142  (23.4s)
[003] lr=0.00500  train loss=3.2689 acc=0.113 | val loss=3.0283 acc=0.147  (22.9s)
[004] lr=0.00500  train loss=3.1831 acc=0.120 | val loss=3.0216 acc=0.148  (23.5s)
[005] lr=0.00500  train loss=3.1407 acc=0.136 | val loss=3.0159 acc=0.146  (22.9s)
[006] lr=0.00500  train loss=3.1014 acc=0.145 | val loss=3.0095 acc=0.152  (23.5s)
[007] lr=0.00450  train loss=3.0789 acc=0.143 | val loss=3.0090 acc=0.152  (23.5s)
[008] lr=0.00450  train loss=3.0554 acc=0.150 | val loss=2.9981 acc=0.150  (23.5s)
[009] lr=0.00450  train loss=3.0360 acc=0.152 | val loss=2.9957 acc=0.166  (22.7s)
[010] lr=0.00450  train loss=3.0181 acc=0.162 | val loss=2.9921 acc=0.159  (26.3s)
[011] lr=0.00450  train loss=3.0042 acc=0.163 | val loss=2.9844 acc=0.164  (23.2s)
[012] lr=0.00405  train loss=2.9970 acc=0.162 | val 

## 자꾸 과적합 아니면 정확도가 15% 밖에 안나옴
- 이유 사진의 픽셀을 뭉텅이로 보는게 아니라 각자 따로 보기 때문에 그런거임 
- 컨볼루션(Conv) + 풀링(MaxPool) 으로 = CNN 3*3 뭐 미런식으로 봐서 귀 형태를 잡는다거나 이런식으로 접근 하면 될듯?!

In [ ]:
# train_ssua_cnn_standalone.py
# ------------------------------------------------------------
# 강아지 종 분류: 순수 NumPy CNN (Conv2d + MaxPool2d + GAP + FC)
# - 메모리-세이프 Conv2d (샘플별 im2col/grad 재계산)
# - 축 안전 벡터화 MaxPool2d (2x2,stride=2 전용)
# - Global Average Pooling으로 일반화↑, 파라미터↓
# - EarlyStopping, Label Smoothing(초기 0), Dropout(0.2), BN1d
# - DataLoader가 (B,C,H,W)로 변환해 CNN에 투입
# ------------------------------------------------------------
import os, sys, time, random, glob
import numpy as np
from PIL import Image

# ---- ssuatensortflow 경로 ----
sys.path.insert(0, os.path.join(os.getcwd(), "Model"))

# ---- 라이브러리 임포트 ----
from ssuatensortflow.tensor import Tensor
from ssuatensortflow.layers import Linear, BatchNorm1d, SGD, dropout
from ssuatensortflow.nn import cross_entropy_with_label_smoothing
from ssuatensortflow.callbacks import EarlyStopping

# ================= 전처리/데이터 유틸 =================
class Compose:
    def __init__(self, transforms): self.transforms = transforms
    def __call__(self, img):
        for t in self.transforms: img = t(img)
        return img

class ToRGBResize:
    def __init__(self, width, height):
        self.w, self.h = width, height
    def __call__(self, img):
        if isinstance(img, str):
            img = Image.open(img)
        elif isinstance(img, np.ndarray):
            arr = img
            if arr.dtype != np.uint8:
                arr = np.clip(arr * (255.0 if arr.max() <= 1.0 else 1.0), 0, 255).astype(np.uint8)
            img = Image.fromarray(arr)
        img = img.convert("RGB").resize((self.w, self.h))
        return np.array(img, dtype=np.float32) / 255.0  # (H,W,3)

class Normalize01:
    def __init__(self, mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)):
        self.mean = np.array(mean, dtype=np.float32)
        self.std  = np.array(std,  dtype=np.float32)
    def __call__(self, img):
        x = img.data if isinstance(img, Tensor) else img
        return (x - self.mean) / self.std

class RandomHorizontalFlip:
    def __init__(self, p=0.5): self.p = p
    def __call__(self, img):
        if isinstance(img, str): return img
        if np.random.rand() < self.p and isinstance(img, np.ndarray):
            return np.fliplr(img)
        return img

class RandomPadCrop:
    def __init__(self, padding=8): self.padding = padding
    def __call__(self, img):
        if isinstance(img, str): return img
        if self.padding <= 0: return img
        h, w, c = img.shape
        pad = self.padding
        padded = np.pad(img, ((pad,pad),(pad,pad),(0,0)), mode="reflect")
        top  = np.random.randint(0, 2*pad)
        left = np.random.randint(0, 2*pad)
        return padded[top:top+h, left:left+w]

def stratified_split(data_root, val_ratio=0.15, seed=42):
    rng = random.Random(seed)
    classes = sorted([d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))])
    cls_to_idx = {c:i for i,c in enumerate(classes)}
    train, val = [], []
    for c in classes:
        files = []
        for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp"):
            files += glob.glob(os.path.join(data_root, c, ext))
        files.sort(); rng.shuffle(files)
        k = int(len(files) * (1 - val_ratio))
        train += [(f, cls_to_idx[c]) for f in files[:k]]
        val   += [(f, cls_to_idx[c]) for f in files[k:]]
    rng.shuffle(train); rng.shuffle(val)
    return train, val

class ImageFolderDataset:
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform
        self.classes = sorted(set([y for _, y in samples]))
        self.num_classes = len(self.classes)
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        path, y = self.samples[idx]
        x = self.transform(path) if self.transform else np.array(Image.open(path).convert("RGB"), dtype=np.float32)/255.0
        if not isinstance(x, Tensor):
            x = Tensor(x.astype(np.float32))  # (H,W,C)
        return x, np.int64(y)

class DataLoader:
    def __init__(self, dataset, batch_size=64, shuffle=True, drop_last=False):
        self.ds, self.bs, self.shuffle, self.drop_last = dataset, batch_size, shuffle, drop_last
    def __iter__(self):
        idxs = list(range(len(self.ds)))
        if self.shuffle: random.shuffle(idxs)
        batch=[]
        for i in idxs:
            batch.append(self.ds[i])
            if len(batch)==self.bs:
                yield self._collate(batch); batch=[]
        if batch and not self.drop_last:
            yield self._collate(batch)
    def __len__(self):
        n = len(self.ds)//self.bs
        if len(self.ds)%self.bs and not self.drop_last: n+=1
        return max(1,n)
    def _collate(self, batch):
        xs, ys = zip(*batch)
        x_np = np.stack([x.data for x in xs], axis=0).astype(np.float32)  # (B,H,W,C)
        x_np = np.transpose(x_np, (0,3,1,2)).copy()  # → (B,C,H,W)
        return Tensor(x_np), np.array(ys, dtype=np.int64)

def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed)

def save_checkpoint(path, state: dict):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    np.savez_compressed(path, **state)

# ================= Conv / Pool (메모리-세이프) =================
def _im2col(x, kh, kw, sh, sw, ph, pw):
    # x: (B,C,H,W) numpy
    B,C,H,W = x.shape
    H_out = (H + 2*ph - kh)//sh + 1
    W_out = (W + 2*pw - kw)//sw + 1
    x_pad = np.pad(x, ((0,0),(0,0),(ph,ph),(pw,pw)), mode='constant')
    cols = np.zeros((B, C, kh, kw, H_out, W_out), dtype=x.dtype)
    for i in range(kh):
        i_lim = i + sh*H_out
        for j in range(kw):
            j_lim = j + sw*W_out
            cols[:, :, i, j, :, :] = x_pad[:, :, i:i_lim:sh, j:j_lim:sw]
    cols = cols.reshape(B, C*kh*kw, H_out*W_out)
    return cols, H_out, W_out

def _col2im(dcols, x_shape, kh, kw, sh, sw, ph, pw):
    B,C,H,W = x_shape
    H_out = (H + 2*ph - kh)//sh + 1
    W_out = (W + 2*pw - kw)//sw + 1
    dcols = dcols.reshape(B, C, kh, kw, H_out, W_out)
    dx_pad = np.zeros((B, C, H+2*ph, W+2*pw), dtype=dcols.dtype)
    for i in range(kh):
        i_lim = i + sh*H_out
        for j in range(kw):
            j_lim = j + sw*W_out
            dx_pad[:, :, i:i_lim:sh, j:j_lim:sw] += dcols[:, :, i, j, :, :]
    return dx_pad[:, :, ph:ph+H, pw:pw+W]

class Conv2d:
    """메모리-세이프 Conv2d (샘플별 im2col)"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True, init='he'):
        kh = kw = kernel_size if isinstance(kernel_size, int) else kernel_size[0]
        self.kh, self.kw = kh, kw
        self.sh = stride if isinstance(stride, int) else stride[0]
        self.sw = stride if isinstance(stride, int) else stride[1]
        self.ph = padding if isinstance(padding, int) else padding[0]
        self.pw = padding if isinstance(padding, int) else padding[1]
        fan_in = in_channels*kh*kw
        limit = np.sqrt(6.0/max(1,fan_in)) if init=='he' else np.sqrt(6.0/(fan_in+out_channels*kh*kw))
        W = np.random.uniform(-limit, limit, size=(out_channels, in_channels, kh, kw)).astype(np.float32)
        self.W = Tensor(W, requires_grad=True)
        self.b = Tensor(np.zeros((1,out_channels,1,1), np.float32), requires_grad=True) if bias else None

    def parameters(self):
        return [p for p in (self.W, self.b) if p is not None]

    def __call__(self, x: Tensor):
        X = x.data  # (B,C,H,W)
        B, Cin, H, W = X.shape
        Cout = self.W.data.shape[0]
        # 출력 크기 계산
        _, H_out, W_out = _im2col(X[:1], self.kh, self.kw, self.sh, self.sw, self.ph, self.pw)
        Y = np.zeros((B, Cout, H_out, W_out), dtype=np.float32)
        W_col = self.W.data.reshape(Cout, -1)  # (Cout, K)

        # 샘플별로 cols 생성 → matmul → 버리기 (피크 메모리↓)
        for b in range(B):
            cols_b, _, _ = _im2col(X[b:b+1], self.kh, self.kw, self.sh, self.sw, self.ph, self.pw)  # (1,K,L)
            cols_b = cols_b[0]  # (K, L)
            Yb = W_col @ cols_b  # (Cout, L)
            if self.b is not None: Yb += self.b.data.reshape(-1,1)
            Y[b] = Yb.reshape(Cout, H_out, W_out)

        out_t = Tensor(
            Y,
            requires_grad=x.requires_grad or self.W.requires_grad or (self.b is not None and self.b.requires_grad),
            _children=(x, self.W, self.b) if self.b is not None else (x, self.W,),
            _op="conv2d_memsafe"
        )

        def _backward():
            if out_t.grad is None: return
            gy = out_t.grad  # (B,Cout,H_out,W_out)
            Cout, Cin, kh, kw = self.W.data.shape
            W_col = self.W.data.reshape(Cout, -1)
            dW = np.zeros_like(self.W.data)
            db = np.zeros((1, Cout, 1, 1), dtype=np.float32) if self.b is not None else None
            dX = np.zeros_like(X) if x.requires_grad else None

            # 샘플별로 cols 재계산 (메모리 절약)
            for b in range(B):
                cols_b, _, _ = _im2col(X[b:b+1], self.kh, self.kw, self.sh, self.sw, self.ph, self.pw)  # (1,K,L)
                cols_b = cols_b[0]                 # (K, L)
                gyb = gy[b].reshape(Cout, -1)     # (Cout, L)

                dW += (gyb @ cols_b.T).reshape(dW.shape)
                if db is not None:
                    db += gyb.sum(axis=1).reshape(1, Cout, 1, 1)
                if dX is not None:
                    dx_cols = W_col.T @ gyb        # (K, L)
                    dX[b:b+1] += _col2im(dx_cols.reshape(1, *dx_cols.shape), X[b:b+1].shape,
                                         self.kh, self.kw, self.sh, self.sw, self.ph, self.pw)

            if self.W.requires_grad:
                self.W.grad = (self.W.grad if self.W.grad is not None else 0) + dW
            if (self.b is not None) and self.b.requires_grad:
                self.b.grad = (self.b.grad if self.b.grad is not None else 0) + db
            if x.requires_grad:
                x.grad = (x.grad if x.grad is not None else 0) + dX

        out_t._backward = _backward
        return out_t

class MaxPool2d:
    """벡터화 MaxPool2d (2x2, stride=2 전용; 축 안전 버전)"""
    def __init__(self, kernel_size=2, stride=2):
        self.kh = self.kw = kernel_size if isinstance(kernel_size,int) else kernel_size[0]
        self.sh = self.sw = stride if isinstance(stride,int) else stride[0]
        assert self.kh==2 and self.kw==2 and self.sh==2 and self.sw==2, "2x2,stride=2만 지원"
        self._mask = None

    def __call__(self, x: Tensor):
        X = x.data  # (B,C,H,W)
        B,C,H,W = X.shape
        H2 = (H//2)*2; W2 = (W//2)*2
        Xc = X[:, :, :H2, :W2]
        Hh, Ww = H2//2, W2//2

        # (B,C,H/2,2,W/2,2)
        Xr = Xc.reshape(B, C, Hh, 2, Ww, 2)

        # 두 축(3,5)에서 한 번에 max → (B,C,H/2,W/2)
        out = Xr.max(axis=(3,5))

        # argmax로 정확한 마스크 구성
        flat = Xr.reshape(B, C, Hh, Ww, 4)   # 2x2 → 4
        arg  = flat.argmax(axis=4)           # (B,C,H/2,W/2) ∈ {0..3}
        mask = np.zeros_like(flat, dtype=bool)
        b = np.arange(B)[:,None,None,None]
        c = np.arange(C)[None,:,None,None]
        h = np.arange(Hh)[None,None,:,None]
        w = np.arange(Ww)[None,None,None,:]
        mask[b, c, h, w, arg] = True
        mask = mask.reshape(B, C, Hh, 2, Ww, 2)

        out_t = Tensor(out, requires_grad=x.requires_grad, _children=(x,), _op="maxpool2d_vec")
        self._mask = (mask, X.shape)

        def _backward():
            if out_t.grad is None or not x.requires_grad: 
                return
            mask, orig_shape = self._mask
            B,C,H,W = orig_shape
            H2 = (H//2)*2; W2 = (W//2)*2
            Hh, Ww = H2//2, W2//2

            grad_out = out_t.grad.reshape(B, C, Hh, 1, Ww, 1)   # (B,C,H/2,1,W/2,1)
            go2 = np.broadcast_to(grad_out, mask.shape)         # (B,C,H/2,2,W/2,2)
            gx_small = np.where(mask, go2, 0.0)

            gx = np.zeros((B,C,H,W), dtype=out_t.grad.dtype)
            gx[:, :, :H2, :W2] = gx_small.reshape(B, C, H2, W2)
            x.grad = (x.grad if x.grad is not None else 0) + gx

        out_t._backward = _backward
        return out_t

# ================= 설정 =================
DATA_ROOT = "data"   # (대문자 M)
IMG_SIZE  = 112
BATCH     = 16
EPOCHS    = 10          # ↑ 안정 모드
LR        = 5e-3        # ↓ 발산 방지
MOM       = 0.9
WD        = 1e-4        # 규제 약간
SMOOTH    = 0.0         # 초반은 0.0으로 신호 세게
PATIENCE  = 12
VAL_RATIO = 0.15
SEED      = 42

seed_everything(SEED)

def build_transforms(split):
    if split == "train":
        return Compose([
            ToRGBResize(IMG_SIZE, IMG_SIZE),
            # RandomPadCrop(padding=8),   # 속도 여유 생기면 켜기
            RandomHorizontalFlip(0.5),
            Normalize01(),
        ])
    else:
        return Compose([
            ToRGBResize(IMG_SIZE, IMG_SIZE),
            Normalize01(),
        ])

if not os.path.isdir(DATA_ROOT):
    raise FileNotFoundError(f"DATA_ROOT가 없습니다: {DATA_ROOT}")

train_files, val_files = stratified_split(DATA_ROOT, val_ratio=VAL_RATIO, seed=SEED)
train_ds = ImageFolderDataset(train_files, transform=build_transforms("train"))
val_ds   = ImageFolderDataset(val_files,   transform=build_transforms("val"))

num_classes = train_ds.num_classes
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, drop_last=False)

# ================= 작은 CNN (GAP 사용) =================
class SmallCNN:
    def __init__(self, in_ch=3, num_classes=10, hidden=256):
        # 앞단 stride=2로 다운샘플 → 메모리/속도 안정
        self.conv1 = Conv2d(in_ch, 16, kernel_size=3, stride=2, padding=1, init='he')
        self.conv2 = Conv2d(16,   32, kernel_size=3, stride=1, padding=1, init='he')
        self.pool1 = MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = Conv2d(32,   64, kernel_size=3, stride=1, padding=1, init='he')
        self.conv4 = Conv2d(64,  128, kernel_size=3, stride=1, padding=1, init='he')
        self.pool2 = MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = None; self.bn1 = None; self.fc2 = None
        self.hidden = hidden
        self.num_classes = num_classes
        self.training = True

    def parameters(self):
        ps = []
        for m in [self.conv1, self.conv2, self.conv3, self.conv4]:
            ps += m.parameters()
        if self.fc1 is not None:
            ps += self.fc1.parameters() + self.bn1.parameters() + self.fc2.parameters()
        return ps

    def train(self): self.training = True
    def eval(self):  self.training = False

    def relu(self, x):
        mask = (x.data > 0).astype(x.data.dtype)
        return x * mask

    def _ensure_fc(self, feat_4d: Tensor):
        B,C,H,W = feat_4d.data.shape
        F = C              # ✅ GAP 쓰니까 채널 수만 입력
        if self.fc1 is None:
            self.fc1 = Linear(F, self.hidden, init="he")
            self.bn1 = BatchNorm1d(self.hidden)
            self.fc2 = Linear(self.hidden, self.num_classes, init="xavier")

    def forward(self, x):  # x: (B,C,H,W)
        y = self.conv1(x); y = self.relu(y)
        y = self.conv2(y); y = self.relu(y)
        y = self.pool1(y)
        y = self.conv3(y); y = self.relu(y)
        y = self.conv4(y); y = self.relu(y)
        y = self.pool2(y)

        # FC 구성은 GAP 전에
        self._ensure_fc(y)

        # ✅ Global Average Pool: (B,C,H,W) -> (B,C)
        B, C, H, W = y.data.shape
        avg = y.data.mean(axis=(2,3))  # (B,C)
        y_gap = Tensor(avg, requires_grad=y.requires_grad, _children=(y,), _op="gap")
        def _bw_gap():
            if y_gap.grad is None or not y.requires_grad: return
            g = y_gap.grad[:, :, None, None] / (H*W)     # (B,C,1,1)
            g = np.broadcast_to(g, (B,C,H,W))
            y.grad = (y.grad if y.grad is not None else 0) + g
        y_gap._backward = _bw_gap

        z = self.fc1(y_gap)
        z = self.bn1(z, train=self.training)
        z = self.relu(z)
        z = dropout(z, p=0.2, train=self.training)   # 0.3 -> 0.2
        z = self.fc2(z)
        return z

# ================= 학습 루프 =================
model = SmallCNN(in_ch=3, num_classes=num_classes, hidden=256)

# 워밍업(FC 생성) + 옵티마이저 준비
x_warm, y_warm = next(iter(train_loader))
_ = model.forward(x_warm)
optimizer = SGD(model.parameters(), lr=LR, momentum=MOM, weight_decay=WD)
early = EarlyStopping(patience=PATIENCE)

def run_one_epoch(loader, train=True):
    if train: model.train()
    else:     model.eval()
    total_loss, total_correct, total_count = 0.0, 0, 0
    for bi, (x,y) in enumerate(loader, 1):
        logits = model.forward(x)
        loss = cross_entropy_with_label_smoothing(logits, y, eps=SMOOTH)
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        total_loss += float(loss.data)
        pred = np.argmax(logits.data, axis=1)
        total_correct += int((pred==y).sum())
        total_count  += y.shape[0]
    return total_loss/max(1,len(loader)), total_correct/max(1,total_count)

best_path = os.path.join("Model", "checkpoints", "best_cnn.npz")
os.makedirs(os.path.dirname(best_path), exist_ok=True)

print(f"Classes: {num_classes}, Train: {len(train_ds)}, Val: {len(val_ds)}")
prev_val = float("inf")
for epoch in range(1, EPOCHS+1):
    t0=time.time()
    tr_loss, tr_acc = run_one_epoch(train_loader, True)
    va_loss, va_acc = run_one_epoch(val_loader, False)

    # plateau 감지는 5epoch 이후부터
    if epoch > 5 and (prev_val - va_loss) < 5e-4:
        optimizer.lr *= 0.9
    prev_val = va_loss

    # 얼리 스탑 체크포인트
    early.update(va_loss,
        lambda: {
            "conv1.W": model.conv1.W.data.copy(),
            "conv1.b": None if model.conv1.b is None else model.conv1.b.data.copy(),
            "conv2.W": model.conv2.W.data.copy(),
            "conv2.b": None if model.conv2.b is None else model.conv2.b.data.copy(),
            "conv3.W": model.conv3.W.data.copy(),
            "conv3.b": None if model.conv3.b is None else model.conv3.b.data.copy(),
            "conv4.W": model.conv4.W.data.copy(),
            "conv4.b": None if model.conv4.b is None else model.conv4.b.data.copy(),
            "fc1.W":   model.fc1.W.data.copy(),
            "fc1.b":   None if model.fc1.b is None else model.fc1.b.data.copy(),
            "bn1.gamma": model.bn1.gamma.data.copy(),
            "bn1.beta":  model.bn1.beta.data.copy(),
            "bn1.running_mean": model.bn1.running_mean.copy(),
            "bn1.running_var":  model.bn1.running_var.copy(),
            "fc2.W":   model.fc2.W.data.copy(),
            "fc2.b":   None if model.fc2.b is None else model.fc2.b.data.copy(),
        },
        lambda sd: None
    )

    try:
        save_checkpoint(best_path, {
            "conv1.W": model.conv1.W.data,
            "conv1.b": None if model.conv1.b is None else model.conv1.b.data,
            "conv2.W": model.conv2.W.data,
            "conv2.b": None if model.conv2.b is None else model.conv2.b.data,
            "conv3.W": model.conv3.W.data,
            "conv3.b": None if model.conv3.b is None else model.conv3.b.data,
            "conv4.W": model.conv4.W.data,
            "conv4.b": None if model.conv4.b is None else model.conv4.b.data,
            "fc1.W":   model.fc1.W.data,
            "fc1.b":   None if model.fc1.b is None else model.fc1.b.data,
            "bn1.gamma": model.bn1.gamma.data,
            "bn1.beta":  model.bn1.beta.data,
            "bn1.running_mean": model.bn1.running_mean,
            "bn1.running_var":  model.bn1.running_var,
            "fc2.W":   model.fc2.W.data,
            "fc2.b":   None if model.fc2.b is None else model.fc2.b.data,
        })
    except Exception:
        pass

    dt=time.time()-t0
    print(f"[{epoch:03d}] lr={optimizer.lr:.5f}  train loss={tr_loss:.4f} acc={tr_acc:.3f} | "
          f"val loss={va_loss:.4f} acc={va_acc:.3f}  ({dt:.1f}s)")

    if early.stopped:
        print(f"Early stopped at epoch {epoch}")
        break

print("Done.")



Classes: 25, Train: 4229, Val: 749
